#  setup

In [1]:
%%file locate_file.py

def locate_file() : 
    working_directory = ''
    for element in __file__.split('/')[:-1] :
        working_directory += element + '/'
    return working_directory

Overwriting locate_file.py


In [2]:
import locate_file
working_directory = locate_file.locate_file()

import os
os.chdir(working_directory)

# convolution.c

In [3]:
%%file convolution.c
#import <math.h>
#include <stdio.h>

void convolution(int *array, int length) {
    int i;
    for(i=0;i<length;i++){
         *(array+i) += 1;
    }
}

Overwriting convolution.c


# compilation

In [4]:
os.system('cd ' + working_directory)
!clang -c convolution.c
!clang -o libconvolution.so -shared convolution.o
!file libconvolution.so

libconvolution.so: Mach-O 64-bit dynamically linked shared library x86_64


# python wrapper module

In [5]:
%%file _convolution.py
import ctypes
import numpy
import locate_file
working_directory = locate_file.locate_file()

_convolution_ = ctypes.cdll.LoadLibrary(working_directory + 'libconvolution.so')

class image_type() : 
    
    def from_param(self,param) : 
        typename = type(param).__name__
        if hasattr(self,'from_'+typename) : 
            return getattr(self,'from_'+typename)(param)
        elif isinstance(param,ctypes.Array) : 
            
            return param
        else : 
            raise TypeError('type %s not supported' %typename)
    
    def from_ndarray(self,param) : 
        c_array = numpy.ascontiguousarray(param)
        pointer = c_array.ctypes
        return pointer
    
    def from_list(self,param) : 
        return self.from_ndarray(param)
    
    def from_tuple(self,param) : 
        return self.from_ndarray(param)
    
image_type_ = image_type()
    
_convolution_.convolution.argtypes = [image_type_,ctypes.c_int]
_convolution_.convolution.restype = ctypes.c_void_p

def convolution(array) : 
    c_array = numpy.ascontiguousarray(array,dtype=numpy.uint32)
    _convolution_.convolution(c_array,len(array))
    return c_array

Overwriting _convolution.py


**numpy.ascontiguousarray** returns a contiguous array in memory (C order)

In [6]:
!python _convolution.py

In [7]:
import _convolution

In [8]:
m = [1,2,3]
m = _convolution.convolution(m)
m

array([2, 3, 4], dtype=uint32)